In [ ]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/PHE Database/2. Coding/2.3. Data Wrangling/Output'
cur_date = '081925'

library(readxl)
library(tidyverse)
library(reshape2)
library(lubridate)
library(oce)
library(tigris)

# reading in files
ohio_tmax_2020_df = read_csv("Input/Ohio_Tmax_2020.csv")
ohio_tmax_2021_df = read_csv('Input/Ohio_Tmax_2021.csv')
ohio_tmax_2022_df = read_csv('Input/Ohio_Tmax_2022.csv')
ohio_PM_2020_df = read_csv('Input/Ohio_PM_2020.csv')
ohio_PM_2021_df = read_csv('Input/Ohio_PM_2021.csv')
ohio_PM_2022_df = read_csv('Input/Ohio_PM_2022.csv')
ohio_tmin_2020_df = read_csv('Input/Ohio_Tmin_2020.csv')
ohio_tmin_2021_df = read_csv('Input/Ohio_Tmin_2021.csv')
ohio_tmin_2022_df = read_csv('Input/Ohio_Tmin_2022.csv')
ohio_HI_2020_df = read_csv('Input/Ohio_HI_2020.csv')
ohio_HI_2021_df = read_csv('Input/Ohio_HI_2021.csv')
ohio_HI_2022_df = read_csv('Input/Ohio_HI_2022.csv')

In [ ]:
head(ohio_tmax_2020_df)

In [ ]:
convert_date = function(df, collection_year, variable){
    # """
    # Creating a function to convert day of the year into a date with the month, day, and year
    # :param (input): dataframe, year the data was collected, variable name
    # :output: 1 df containing the coordinates, value, date of collection, variable name
    # """
    
    new_df = df %>%
        mutate(Month = month(as_date(DOY)), Day = mday(as_date(DOY)), Year = collection_year,
           Date = format(make_date(month = Month, day = Day, year = Year), format = "%m/%d/%Y"),
           # adding in the variable's name
           VariableName = variable) %>%
        select(-c('DOY', "Month", "Day", "Year")) 

    return(new_df)
    }

# calling fn
converted_tmax_2020_df = convert_date(ohio_tmax_2020_df, 2020, 'Maximum Temperature')
converted_tmax_2021_df = convert_date(ohio_tmax_2021_df, 2021, 'Maximum Temperature')
converted_tmax_2022_df = convert_date(ohio_tmax_2022_df, 2022, 'Maximum Temperature')
converted_HI_2020_df = convert_date(ohio_HI_2020_df, 2020, 'Heat Index')
converted_HI_2021_df = convert_date(ohio_HI_2021_df, 2021, 'Heat Index')
converted_HI_2022_df = convert_date(ohio_HI_2022_df, 2022, 'Heat Index')
converted_tmin_2020_df = convert_date(ohio_tmin_2020_df, 2020, 'Minimum Temperature')
converted_tmin_2021_df = convert_date(ohio_tmin_2021_df, 2021, 'Minimum Temperature')
converted_tmin_2022_df = convert_date(ohio_tmin_2022_df, 2022, 'Minimum Temperature')
# moving around these col names here to combine dfs below
converted_PM_2020_df = convert_date(ohio_PM_2020_df, 2020, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)
converted_PM_2021_df = convert_date(ohio_PM_2021_df, 2021, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)
converted_PM_2022_df = convert_date(ohio_PM_2022_df, 2022, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)

head(converted_tmax_2020_df)

In [ ]:
convert_coordinates = function(df){
    # """
    # Creating a function to convert UTM coordinates to latitude and longitude
    # :param (input): dataframe
    # :output: dataframe
    # """

    coordinates_df = df %>%
        # zone 14 is for Ohio
        mutate(Latitude = utm2lonlat(easting = x, northing = y, zone = 14)[[1]],
              Longitude = utm2lonlat(easting = x, northing = y, zone = 14)[[2]]) %>%
        select(-c('x', 'y')) 
    
    return(coordinates_df)
}

# calling fn
final_tmax_2020_df = convert_coordinates(converted_tmax_2020_df)
final_tmax_2021_df = convert_coordinates(converted_tmax_2021_df)
final_tmax_2022_df = convert_coordinates(converted_tmax_2022_df)
final_HI_2020_df = convert_coordinates(converted_HI_2020_df)
final_HI_2021_df = convert_coordinates(converted_HI_2021_df)
final_HI_2022_df = convert_coordinates(converted_HI_2022_df)
final_tmin_2020_df = convert_coordinates(converted_tmin_2020_df)
final_tmin_2021_df = convert_coordinates(converted_tmin_2021_df)
final_tmin_2022_df = convert_coordinates(converted_tmin_2022_df)

head(final_tmax_2020_df)

Now breaking these data up into the dataframes that they will be stored in for the database. We'll have separate dataframes for each year. 

In [ ]:
twenty20_df = rbind(converted_PM_2020_df, final_HI_2020_df, final_tmax_2020_df, final_tmin_2020_df)
twenty21_df = rbind(converted_PM_2021_df, final_HI_2021_df, final_tmax_2021_df, final_tmin_2021_df)
twenty22_df = rbind(converted_PM_2022_df, final_HI_2022_df, final_tmax_2022_df, final_tmin_2022_df)

In [ ]:
split_dfs = function(df){
    # """
    # Creating a function to add in georaphic id and variable ids
    # :param (input): dataframe
    # :output: a list of dataframes for geography, variable, value
    # """

    new_df = df %>%
        mutate(Num = 1:n(), GeographyWID = paste0('GEOW', Num), ValueID = paste0('Val', Num),
              VariableID = ifelse(VariableName == "PM2.5", 'PM',
                                      ifelse(VariableName == "Maximum Temperature",'MaxT',
                                          ifelse(VariableName == "Minimum Temperature", 'MinT',
                                              ifelse(VariableName == "Heat Index", 'HI', NA))))) %>%
        select(-Num)

    # creating the geography weather df
    geo_df = unique(new_df[,c(6,4,5)])

    # creating the variable df
    var_df = unique(new_df[,c(8,3)]) %>%
        # adding in units
        mutate(DataUnitName = ifelse(VariableName == "PM2.5", 'mcg/m^3',
                                      ifelse(VariableName == "Maximum Temperature",'Fahrenheit',
                                          ifelse(VariableName == "Minimum Temperature", 'Fahrenheit',
                                              ifelse(VariableName == "Heat Index", 'Fahrenheit', NA)))))

    # value df
    value_df = new_df[,c(7,8,6,2,1)] 

    return(list(geo_df, var_df, value_df))
}

# calling fn
split_twenty20_dfs = split_dfs(twenty20_df)
split_twenty21_dfs = split_dfs(twenty21_df)
split_twenty22_dfs = split_dfs(twenty22_df)

In [ ]:
# viewing
head(split_twenty20_dfs[[1]])
head(split_twenty20_dfs[[2]])
head(split_twenty20_dfs[[3]])

Adding in additional geography data like census tract, county name, and the FIPS code to the geography dataframes.

In [ ]:
# download shapefile from the tigris (census package)
ohio_county_shapefile = data.frame(tracts(state = "Ohio", year = 2020)[,c(2,4,5,11,12)]) %>%
    rename(FIPS = GEOID, CensusTract = NAME, Latitude = INTPTLON, Longitude = INTPTLAT) %>%
    select(-geometry) %>%
    mutate(Latitude = as.numeric(Latitude), Longitude = as.numeric(Longitude))

head(ohio_county_shapefile)

Finding the min and max latitude and longitude coordinates for each census tract in order to figure out what census tract each pair of coordinates reside in.

In [ ]:
length(unique(ohio_county_shapefile$CensusTract))

Ohio actually has 2,952 census tracts, meaning we're missing about 400 from the tracts that were downloaded from the `tigris` package.

In [ ]:
min_max_coord_df = ohio_county_shapefile %>%
    group_by(CensusTract) %>%
    summarize(min_long = min(Longitude), max_long = max(Longitude),
            min_lat = min(Latitude), max_lat = max(Latitude))

head(min_max_coord_df)

In [ ]:
geography_2020_df = head(split_twenty20_dfs[[1]]) 

head(geography_2020_df)